# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from datetime import datetime

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.29,67,13,4.74,SH,1701122661
1,1,qarazhal,48.0078,70.7903,5.74,53,100,5.68,KZ,1701122790
2,2,yashkul',46.1711,45.3435,4.26,72,25,4.63,RU,1701122790
3,3,utrik,11.2278,169.8474,27.65,74,73,9.23,MH,1701122790
4,4,stanley,54.8680,-1.6985,5.29,92,75,6.17,GB,1701122704



---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5,
    frame_width = 800,
    frame_height = 600,
    alpha = 0.75)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 29) &
                        (city_data_df["Max Temp"] > 18)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,hawaiian paradise park,19.5933,-154.9731,25.73,79,0,3.60,US,1701122666
204,204,freetown,8.4840,-13.2299,26.83,79,0,3.03,SL,1701122845
226,226,fortuna,40.5982,-124.1573,18.29,44,0,0.89,US,1701122853
240,240,inhambane,-23.8650,35.3833,23.08,87,0,3.06,MZ,1701122857
271,271,chandler,33.3062,-111.8412,24.08,27,0,2.57,US,1701122672



### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity})
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,hawaiian paradise park,US,19.5933,-154.9731,79,
204,freetown,SL,8.4840,-13.2299,79,
226,fortuna,US,40.5982,-124.1573,44,
240,inhambane,MZ,-23.8650,35.3833,87,
271,chandler,US,33.3062,-111.8412,27,
302,pacific grove,US,36.6177,-121.9166,29,
363,ahvaz,IR,31.3203,48.6693,88,
433,toliara,MG,-23.3500,43.6667,89,
462,chiredzi,ZW,-21.0500,31.6667,37,
471,medina,SA,24.4686,39.6142,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat, lon from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address1 = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address1.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
freetown - nearest hotel: Formal Leona hotel
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
inhambane - nearest hotel: Hotel de Inhambane
chandler - nearest hotel: Crowne Plaza - San Marcos Golf Resort
pacific grove - nearest hotel: Pacific Grove Inn
ahvaz - nearest hotel: هتل پارس
toliara - nearest hotel: Ambary
chiredzi - nearest hotel: Nesbitt Arms Hotel
medina - nearest hotel: The Oberoi Madina
taoudenni - nearest hotel: No hotel found
maun - nearest hotel: Center Lodge Conference Center
orange cove - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,hawaiian paradise park,US,19.5933,-154.9731,79,No hotel found
204,freetown,SL,8.4840,-13.2299,79,Formal Leona hotel
226,fortuna,US,40.5982,-124.1573,44,Comfort Inn & Suites Redwood Country
240,inhambane,MZ,-23.8650,35.3833,87,Hotel de Inhambane
271,chandler,US,33.3062,-111.8412,27,Crowne Plaza - San Marcos Golf Resort
302,pacific grove,US,36.6177,-121.9166,29,Pacific Grove Inn
363,ahvaz,IR,31.3203,48.6693,88,هتل پارس
433,toliara,MG,-23.3500,43.6667,89,Ambary
462,chiredzi,ZW,-21.0500,31.6667,37,Nesbitt Arms Hotel
471,medina,SA,24.4686,39.6142,22,The Oberoi Madina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color="City",
    scale=0.5,
    frame_width = 800,
    frame_height = 600,
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)